# 🫀 Clinical-Grade Multimodal ECG Training Pipeline  
<span style="color:red">by Ridwan Oladipo, MD | Medical AI Specialist</span>  

Production-ready training pipeline for **12-lead ECG classification**, implementing a **ResNet-1D + tabular fusion network** with:  

- **ResNet-1D signal branch** → temporal P–QRS–T wave & rhythm morphology modeling  
- **Clinical metadata branch** → HR/HRV + age/sex + device harmonization  
- **Late fusion** → integrated ECG + tabular decision space
- **Binary cross-entropy loss** for multilabel setting  
- **Recall-optimized callbacks** → early stopping & checkpointing to maximize **myocardial infarction sensitivity**  
- **Reproducible training** with fixed seeds & official PTB-XL stratified folds  (preventing patient leakage)

🚀 Trains on **~17k+ ECGs** with structured logging & TensorBoard monitoring.  
>⚕️ **Clinically-aligned optimization** — tuning for **sensitivity and NPV in myocardial infarction detection**, the metrics that matter most in cardiology.

## 🧩Environment Setup and Data Loading

In [1]:
# Essential libraries for deep learning and model training
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv1D, BatchNormalization, Activation, Add
from tensorflow.keras.layers import MaxPooling1D, GlobalAveragePooling1D, Dropout, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

# For monitoring and evaluation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Load Preprocessed Data
base_dir = "/kaggle/input/ecg-preprocessed"
all_signals = np.load(f"{base_dir}/all_signals.npy", allow_pickle=True)
y_labels = np.load(f"{base_dir}/y_labels.npy", allow_pickle=True)
all_features = pd.read_parquet(f"{base_dir}/all_features.parquet")
model_df_with_labels = pd.read_parquet(f"{base_dir}/model_df_with_labels.parquet")

# Reproducibility
np.random.seed(42)
tf.random.set_seed(42)
print("Random seeds set for reproducibility")

print("=== Training Environment Initialized ===")
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

print(f"\n=== Preprocessed Data Verification ===")
print(f"Signals shape: {all_signals.shape}")
print(f"Features shape: {all_features.shape}")
print(f"Labels shape: {y_labels.shape}")
print(f"Classes: {y_labels.shape[1]}")

2025-09-24 20:22:47.360025: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758745367.382678      93 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758745367.389419      93 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Random seeds set for reproducibility
=== Training Environment Initialized ===
TensorFlow version: 2.18.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

=== Preprocessed Data Verification ===
Signals shape: (21837, 1000, 12)
Features shape: (21837, 190)
Labels shape: (21837, 5)
Classes: 5


## 🔀Train/Test Split

In [2]:
# Using PTB-XL Official strat_fold for Train/Test Split
print("\n=== Using PTB-XL Official strat_fold for Train/Test Split ===")

train_idx = model_df_with_labels['strat_fold'] < 9  # folds 1–8 = train
test_idx = model_df_with_labels['strat_fold'] >= 9  # folds 9–10 = test

X_ecg_train, X_ecg_test = all_signals[train_idx], all_signals[test_idx]
X_tab_train, X_tab_test = all_features.loc[train_idx], all_features.loc[test_idx]
y_train, y_test = y_labels[train_idx], y_labels[test_idx]

print(f"✓ Training set: {len(X_ecg_train):,} samples")
print(f"✓ Test set: {len(X_ecg_test):,} samples")

# Class Distribution Verification
class_names = ['NORM', 'MI', 'STTC', 'CD', 'HYP']
train_class_dist = y_train.mean(axis=0)
test_class_dist = y_test.mean(axis=0)

print("\n=== Class Distribution Verification ===")
for i, cls in enumerate(class_names):
    diff = abs(train_class_dist[i] - test_class_dist[i])
    print(f"{cls}: Train {train_class_dist[i]:.3f} | Test {test_class_dist[i]:.3f} | Diff {diff:.3f}")


=== Using PTB-XL Official strat_fold for Train/Test Split ===
✓ Training set: 17,441 samples
✓ Test set: 4,396 samples

=== Class Distribution Verification ===
NORM: Train 0.436 | Test 0.437 | Diff 0.001
MI: Train 0.252 | Test 0.250 | Diff 0.002
STTC: Train 0.240 | Test 0.240 | Diff 0.000
CD: Train 0.224 | Test 0.226 | Diff 0.002
HYP: Train 0.122 | Test 0.121 | Diff 0.000


## 🏗️ ResNet-1D Architecture for ECG Signal Processing

In [3]:
def ResNet1D_block(X, filters, kernel_size=7, stride=1):
    """
    ResNet 1D block with skip connection for ECG signal processing

    This architecture is designed for temporal sequence modeling
    in ECG signals, capturing both local and global cardiac patterns.

    Args:
        X: Input tensor
        filters: Number of convolutional filters
        kernel_size: Convolution kernel size (default 7 for ECG)
        stride: Convolution stride

    Returns:
        Output tensor with residual connection
    """
    X_shortcut = X

    # First convolutional component
    X = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    # Second convolutional component
    X = Conv1D(filters=filters, kernel_size=kernel_size, strides=1, padding='same')(X)
    X = BatchNormalization()(X)

    # Skip connection handling for dimension matching
    if stride > 1 or X_shortcut.shape[-1] != filters:
        X_shortcut = Conv1D(filters=filters, kernel_size=1, strides=stride, padding='same')(X_shortcut)
        X_shortcut = BatchNormalization()(X_shortcut)

    # Add residual connection and activate
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X


def create_resnet1d_multimodal_model(ecg_shape, tab_shape, n_classes=5):
    """
    Build a ResNet-1D based multimodal model for ECG + tabular data classification

    This architecture follows cardiology-informed design principles:
    - Multi-scale temporal feature extraction for rhythm analysis
    - Hierarchical feature learning for morphological pattern recognition
    - Clinical metadata integration for comprehensive assessment
    - Multi-label output for clinical scenarios

    Args:
        ecg_shape: Shape of ECG signal input (timesteps, leads)
        tab_shape: Number of tabular features
        n_classes: Number of output diagnostic classes

    Returns:
        Compiled Keras model ready for training
    """

    # ECG SIGNAL BRANCH
    ecg_input = Input(shape=ecg_shape, name='ecg_input')

    # Initial convolutional layer - captures basic ECG waveforms
    X = Conv1D(filters=64, kernel_size=7, strides=2, padding='same')(ecg_input)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPooling1D(pool_size=3, strides=2, padding='same')(X)

    # ResNet block stack 1: Fine-grained pattern detection (P-QRS-T waves)
    X = ResNet1D_block(X, filters=64, kernel_size=5)
    X = ResNet1D_block(X, filters=64, kernel_size=5)

    # ResNet block stack 2: Intermediate pattern recognition (ST segments, intervals)
    X = ResNet1D_block(X, filters=128, kernel_size=5, stride=2)
    X = ResNet1D_block(X, filters=128, kernel_size=5)

    # ResNet block stack 3: High-level rhythm and morphology patterns
    X = ResNet1D_block(X, filters=256, kernel_size=3, stride=2)
    X = ResNet1D_block(X, filters=256, kernel_size=3)

    # Global feature aggregation
    X = GlobalAveragePooling1D()(X)
    X = Dense(128, activation='relu')(X)
    X = Dropout(0.5)(X)  # Prevent overfitting on ECG patterns
    ecg_output = Dense(64, activation='relu')(X)

    # TABULAR FEATURE BRANCH
    tab_input = Input(shape=(tab_shape,), name='tab_input')

    # Clinical metadata processing network
    Y = Dense(128, activation='relu')(tab_input)
    Y = BatchNormalization()(Y)
    Y = Dropout(0.3)(Y)

    Y = Dense(64, activation='relu')(Y)
    Y = BatchNormalization()(Y)
    Y = Dropout(0.3)(Y)

    tab_output = Dense(32, activation='relu')(Y)

    # MULTIMODAL FUSION
    # Combine ECG signal features with clinical metadata
    combined = concatenate([ecg_output, tab_output], name='multimodal_fusion')

    # Final classification layers
    Z = Dense(64, activation='relu')(combined)
    Z = BatchNormalization()(Z)
    Z = Dropout(0.3)(Z)

    Z = Dense(32, activation='relu')(Z)

    # Multi-label output with sigmoid activation for independent class probabilities
    main_output = Dense(n_classes, activation='sigmoid', name='main_output')(Z)

    # MODEL COMPILATION
    model = Model(inputs=[ecg_input, tab_input], outputs=main_output)

    # Compile with clinically-appropriate metrics
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',  # Appropriate for multi-label classification
        metrics=[
            'accuracy',
            tf.keras.metrics.AUC(multi_label=True, name='auc'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.Precision(name='precision')
        ]
    )

    return model


print("=== ResNet-1D Multimodal Architecture Defined ===")
print("✓ ECG signal processing branch: 1D ResNet with temporal modeling")
print("✓ Tabular feature branch: Dense network for clinical metadata")
print("✓ Multimodal fusion: Late fusion of ECG and clinical features")
print("✓ Multi-label output: Sigmoid activation for independent predictions")

=== ResNet-1D Multimodal Architecture Defined ===
✓ ECG signal processing branch: 1D ResNet with temporal modeling
✓ Tabular feature branch: Dense network for clinical metadata
✓ Multimodal fusion: Late fusion of ECG and clinical features
✓ Multi-label output: Sigmoid activation for independent predictions


## 📋 Model Instantiation and Architecture Summary

In [4]:
# Create the multimodal model
print("=== Creating Multimodal ECG Classification Model ===")

model = create_resnet1d_multimodal_model(
    ecg_shape=(X_ecg_train.shape[1], X_ecg_train.shape[2]),
    tab_shape=X_tab_train.shape[1],
    n_classes=5
)

# Display model architecture
print(f"\n=== Model Architecture Summary ===")
print(f"ECG input shape: {(X_ecg_train.shape[1], X_ecg_train.shape[2])}")
print(f"Tabular input shape: {X_tab_train.shape[1]} features")
print(f"Output classes: 5 (NORM, MI, STTC, CD, HYP)")

model.summary()

# Calculate model parameters
total_params = model.count_params()
trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])

print(f"\n=== Model Complexity Analysis ===")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model memory footprint: ~{total_params * 4 / (1024 ** 2):.1f} MB")

=== Creating Multimodal ECG Classification Model ===


I0000 00:00:1758745373.245864      93 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0



=== Model Architecture Summary ===
ECG input shape: (1000, 12)
Tabular input shape: 190 features
Output classes: 5 (NORM, MI, STTC, CD, HYP)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ ecg_input           │ (None, 1000, 12)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 500, 64)   │      5,440 │ ecg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 500, 64)   │        256 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 500, 64)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 250, 64)   │          0 │ activation[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 250, 64)   │     20,544 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 250, 64)   │        256 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 250, 64)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 250, 64)   │     20,544 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 250, 64)   │        256 │ conv1d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 250, 64)   │          0 │ batch_normalizat… │
│                     │                   │            │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 250, 64)   │          0 │ add[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 250, 64)   │     20,544 │ activation_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 250, 64)   │        256 │ conv1d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 250, 64)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 250, 64)   │     20,544 │ activation_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 250, 64)   │        256 │ conv1d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 250, 64)   │          0 │ batch_normalizat… │
│                     │                   │            │ activation_2[0][… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 1,199,717 (4.58 MB)

 Trainable params: 1,194,725 (4.56 MB)

 Non-trainable params: 4,992 (19.50 KB)


=== Model Complexity Analysis ===
Total parameters: 1,199,717
Trainable parameters: 1,194,725
Model memory footprint: ~4.6 MB


## ⚙️ Training Configuration and Callbacks

In [5]:
# Create directories for model artifacts
checkpoint_dir = './model_checkpoints'
log_dir = './training_logs'

for directory in [checkpoint_dir, log_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")


# Define training callbacks
def create_training_callbacks():
    """
    Create comprehensive callbacks for clinical-grade model training

    These callbacks ensure:
    - Early stopping to prevent overfitting
    - Model checkpointing for best performance preservation
    - Learning rate reduction for convergence optimization
    - Training monitoring and visualization
    """

    callbacks = [
        # Early stopping based on recall
        EarlyStopping(
            monitor='val_recall',
            patience=10,
            restore_best_weights=True,
            mode='max',
            verbose=1
        ),

        # Save best model based on recall
        ModelCheckpoint(
            filepath=os.path.join(checkpoint_dir, 'best_recall_model_{epoch:02d}_{val_recall:.4f}.keras'),
            save_best_only=True,
            monitor='val_recall',
            mode='max',
            verbose=1
        ),

        # Save best model based on precision
        ModelCheckpoint(
            filepath=os.path.join(checkpoint_dir, 'best_precision_model_{epoch:02d}_{val_precision:.4f}.keras'),
            save_best_only=True,
            monitor='val_precision',
            mode='max',
            verbose=1
        ),

        # Learning rate reduction for fine-tuning
        ReduceLROnPlateau(
            monitor='val_recall',
            factor=0.5,
            patience=5,
            mode='max',
            min_lr=0.00001,
            verbose=1
        ),

        # TensorBoard logging for training visualization
        TensorBoard(
            log_dir=os.path.join(log_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S")),
            histogram_freq=1,
            write_graph=True,
            write_images=False,
            update_freq='epoch'
        )
    ]

    return callbacks


# Create callbacks
training_callbacks = create_training_callbacks()

print("=== Training Configuration Complete ===")
print("✓ Early stopping: Monitor validation recall with patience=10")
print("✓ Model checkpoints: Save best recall and precision models")
print("✓ Learning rate reduction: Adaptive LR with plateau detection")
print("✓ TensorBoard logging: Comprehensive training monitoring")

Created directory: ./model_checkpoints
Created directory: ./training_logs
=== Training Configuration Complete ===
✓ Early stopping: Monitor validation recall with patience=10
✓ Model checkpoints: Save best recall and precision models
✓ Learning rate reduction: Adaptive LR with plateau detection
✓ TensorBoard logging: Comprehensive training monitoring


## 🚀Training Execution

In [6]:
# Training hyperparameters
EPOCHS = 40
BATCH_SIZE = 64
VALIDATION_SPLIT = 0.2

print("=" * 60)
print("INITIATING MODEL TRAINING")
print("=" * 60)

print(f"\n=== Training Parameters ===")
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Validation split: {VALIDATION_SPLIT}")
print(f"Optimizer: Adam (lr=0.001)")
print(f"Loss function: Binary crossentropy (multi-label)")

print(f"\n=== Training Data Summary ===")
print(f"Training samples: {len(X_ecg_train):,}")
print(f"Validation samples: {int(len(X_ecg_train) * VALIDATION_SPLIT):,}")
print(f"Features per sample: {X_tab_train.shape[1]:,}")

# Start training with comprehensive monitoring
print(f"\n=== Training Progress ===")
start_time = datetime.datetime.now()

history = model.fit(
    [X_ecg_train, X_tab_train], y_train,
    validation_split=VALIDATION_SPLIT,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=training_callbacks,
    verbose=1,
    shuffle=True  # Ensure data is shuffled each epoch
)

end_time = datetime.datetime.now()
training_duration = end_time - start_time

print(f"\n=== Training Completed ===")
print(f"Total training time: {training_duration}")
print(f"Average time per epoch: {training_duration / len(history.epoch)}")
print(f"Final epoch: {len(history.epoch)}")

INITIATING MODEL TRAINING

=== Training Parameters ===
Epochs: 30
Batch size: 64
Validation split: 0.2
Optimizer: Adam (lr=0.001)
Loss function: Binary crossentropy (multi-label)

=== Training Data Summary ===
Training samples: 17,441
Validation samples: 3,488
Features per sample: 190

=== Training Progress ===
Epoch 1/30


I0000 00:00:1758745395.938780     145 service.cc:148] XLA service 0x791d1c002f70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758745395.939477     145 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1758745397.808197     145 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-09-24 20:23:20.920735: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng12{k11=2} for conv (f32[64,128,1,125]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,128,1,125]{3,2,1,0}, f32[128,128,1,5]{3,2,1,0}, f32[128]{0}), window={size=1x5 pad=0_0x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-09-24 20:

216/218 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.4059 - auc: 0.6798 - loss: 0.5883 - precision: 0.4340 - recall: 0.5022
Epoch 1: val_recall improved from -inf to 0.60603, saving model to ./model_checkpoints/best_recall_model_01_0.6060.keras

Epoch 1: val_precision improved from -inf to 0.66495, saving model to ./model_checkpoints/best_precision_model_01_0.6650.keras
218/218 ━━━━━━━━━━━━━━━━━━━━ 45s 56ms/step - accuracy: 0.4077 - auc: 0.6810 - loss: 0.5866 - precision: 0.4359 - recall: 0.5027 - val_accuracy: 0.5858 - val_auc: 0.8345 - val_loss: 0.4371 - val_precision: 0.6650 - val_recall: 0.6060 - learning_rate: 0.0010
Epoch 2/30
217/218 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6416 - auc: 0.8547 - loss: 0.3637 - precision: 0.7284 - recall: 0.6003
Epoch 2: val_recall did not improve from 0.60603

Epoch 2: val_precision did not improve from 0.66495
218/218 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.6417 - auc: 0.8548 - loss: 0.3636 - precision: 0.7285 - recall: 0.6005 